<center><img src="https://images.squarespace-cdn.com/content/v1/5d5ebe0290b74100011fd096/1596042157547-KXGXKTJQ4KH2DDXM5726/FourthBrain%28noDescritor%29Logo.png?format=1500w" width=200 height=200 /></center>

# Environment Class

## Import Libraries

In [1]:
# import gym
import gym
from gym import Env


/home/cim/anaconda3/envs/packing_env/lib/python3.7/site-packages/ale_py/roms/__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()


## Space

In [2]:
import numpy as np
from functools import reduce
import copy, time

class Box(object):
    def __init__(self, x, y, lx, ly):
        self.x = x
        self.y = y
        self.lx = lx
        self.ly = ly
        
    def standardize(self):
        return tuple([self.x, self.y, self.lx, self.ly])

class Space(object):
    def __init__(self, length=10, height=10):
        self.plain_size = np.array([length, height])
        self.plain = np.zeros(shape=(length, height), dtype=np.int32)
        self.boxes = []
        self.flags = [] # record rotation information
        self.height = height

    def print_area_graph(self): # print_height_graph
        print(self.plain)

    #def get_height_graph(self):
    #    plain = np.zeros(shape=self.plain_size[:], dtype=np.int32)
    #    for box in self.boxes:
    #        plain = self.update_height_graph(plain, box)
    #    return plain

    #@staticmethod
    #def update_height_graph(plain, box):
    #    plain = copy.deepcopy(plain)
    #    le = box.lx
    #    ri = box.lx + box.x
    #    up = box.ly
    #    do = box.ly + box.y
        #max_h = np.max(plain[le:ri, up:do])
        #max_h = max(max_h, box.ly + box.y)
        #plain[le:ri, up:do] = max_h
     #   return plain

    def get_box_list(self):
        vec = list()
        for box in self.boxes:
            vec += box.standardize()
        return vec

    def get_plain():
        return copy.deepcopy(self.plain)

    def get_action_space(self):
        return self.plain_size[0] * self.plain_size[1]

    def check_box(self, plain, x, y, lx, ly): # x,y location of box, lx,ly size of plane (from bottom left corner)
        if lx+x > self.plain_size[0] or ly+y > self.plain_size[1]:
            return -1
        if lx < 0 or ly < 0:
            return -1
        
        rec = plain[lx:lx+x, ly:ly+y]
        r00 = rec[0,0]
        r10 = rec[x-1,0]
        r01 = rec[0,y-1]
        r11 = rec[x-1,y-1]
        rm = max(r00,r10,r01,r11)
        sc = int(r00==rm)+int(r10==rm)+int(r01==rm)+int(r11==rm)
        if sc < 3:
            return -1
        # get the max height
        max_h = np.max(rec)
        # check area and corner
        max_area = np.sum(rec==max_h)
        area = x * y

        # check boundary
        assert max_h >= 0
        if max_h + y > self.height:
            return -1
     
        if max_area/area > 0.95:
            return max_h
        if rm == max_h and sc == 3 and max_area/area > 0.85:
            return max_h
        if rm == max_h and sc == 4 and max_area/area > 0.50:
            return max_h

        return -1
    
    def get_ratio(self):
        vo = reduce(lambda x, y: x+y, [box.x * box.y for box in self.boxes], 0.0) # * box.z 
        mx = self.plain_size[0] * self.plain_size[1]# * self.plain_size[2]
        ratio = vo / mx
        assert ratio <= 1.0
        return ratio

    def idx_to_position(self, idx):
        lx = idx // self.plain_size[1]
        ly = idx % self.plain_size[1]
        return lx, ly

    def position_to_index(self, position):
        assert len(position) == 2
        assert position[0] >= 0 and position[1] >= 0
        assert position[0] < self.plain_size[0] and position[1] < self.plain_size[1]
        return position[0] * self.plain_size[1] + position[1]

    def drop_box(self, box_size, idx, flag):
        lx = self.idx_to_position(idx) #, ly
        
        if not flag:
            x = box_size[0]
            #y = box_size[1]
        else:
            x = box_size[1]
            #y = box_size[0]
        #z = box_size[2]
        plain = self.plain
        new_h = self.check_box(plain, x, lx, y) #self.check_box(plain, x, y, lx, ly, z)
        if new_h != -1:
            self.boxes.append(Box(x, y, z, lx, ly, new_h)) # record rotated box
            self.flags.append(flag)
            self.plain = self.update_height_graph(plain, self.boxes[-1])
            self.height = max(self.height, new_h + z)
            return True
        
        return False
    

## Environment

In [3]:
# 
import gym
from gym import Env

# Create class 2D
class Packing2D(Env):
    """
    Pakcing2D is 2D environment class that contains boxes and bin (container). 
    In this class we should have following function:
    - init: is a constructor that construct the class\=
    - 
    """
    # building constructor
    def __init__(self, box_creator=None, container_size = (20, 20),
                 box_set = None, data_name = None, test = False,
                 data_type = 'depen', enable_rotation=False, **kwags):
        self.box_creator = box_creator
        self.bin_size = container_size
        self.area = int(self.bin_size[0] * self.bin_size[1])
        self.space = Space(*self.bin_size)
        self.can_rotate = enable_rotation

        if not test and box_creator is None:
            assert box_set is not None
            if data_type == 'sample':
                print('using random data')
                self.box_creator = RandomBoxCreator(box_set)
            elif data_type == 'depen':
                low = list(box_set[0])
                up = list(box_set[-1])
                low.extend(up)
                print(low)
                self.box_creator = CuttingBoxCreator(container_size, low, self.can_rotate)
            elif data_type == 'md':
                print('using md data')
                self.box_creator = MDlayerBoxCreator(container_size, [box_set[0][0], box_set[-1][0]])
            assert isinstance(self.box_creator, BoxCreator)

        if test:
            self.box_creator = LoadBoxCreator(data_name)

        self.act_len = self.area * (1+self.can_rotate)
        self.obs_len = self.area * (1+3)
        self.action_space = gym.spaces.Discrete(self.act_len)
        self.observation_space = gym.spaces.Box(low=0.0, high=self.space.height, shape=(self.obs_len,))
        

    def get_box_ratio(self):
        coming_box = self.next_box
        return (coming_box[0] * coming_box[1] * coming_box[2]) / (self.space.plain_size[0] * self.space.plain_size[1] * self.space.plain_size[2])


    def get_box_plain(self):
        x_plain = np.ones(self.space.plain_size[:2], dtype=np.int32) * self.next_box[0]
        y_plain = np.ones(self.space.plain_size[:2], dtype=np.int32) * self.next_box[1]
        z_plain = np.ones(self.space.plain_size[:2], dtype=np.int32) * self.next_box[2]
        return (x_plain, y_plain, z_plain)

    def reset(self):
        self.box_creator.reset()
        self.space = Space(*self.bin_size)
        self.box_creator.generate_box_size()
        return self.cur_observation

    @property
    def cur_observation(self):
        hmap = self.space.plain
        # mask = self.get_possible_position()
        size = self.get_box_plain()
        return np.reshape(np.stack((hmap,  *size)), newshape=(-1,))

    @property
    def next_box(self):
        return self.box_creator.preview(1)[0]

    def get_possible_position(self, plain=None):
        x = self.next_box[0]
        y = self.next_box[1]
        z = self.next_box[2]

        if plain is None:
            plain = self.space.plain

        width = self.space.plain_size[0]
        length = self.space.plain_size[1]

        action_mask = np.zeros(shape=(width, length), dtype=np.int32)
        
        for i in range(width-x+1):
            for j in range(length-y+1):
                if self.space.check_box(plain, x, y, i, j, z) >= 0:
                    action_mask[i, j] = 1

        if action_mask.sum() == 0:
            action_mask[:, :] = 1
        
        return action_mask

    def step(self, action):
        idx = action[0]
        flag = False
        # check whether rotate the box
        if idx > self.area:
            assert self.can_rotate
            idx = idx - self.area
            flag = True
        succeeded = self.space.drop_box(self.next_box, idx, flag)

        if not succeeded:
            reward = 0.0
            done = True
            info = {'counter':len(self.space.boxes), 'ratio':self.space.get_ratio(), 'mask':np.ones(shape=self.act_len)}
            return self.cur_observation, reward, done, info

        box_ratio = self.get_box_ratio()

        self.box_creator.drop_box() # remove current box from the list
        self.box_creator.generate_box_size() # add a new box to the list

        plain = self.space.plain

        reward = box_ratio * 10
        done = False
        info = dict()
        info['counter'] = len(self.space.boxes)
        info['ratio'] = self.space.get_ratio()
        # info['mask'] = self.get_possible_position().reshape((-1,))
        return self.cur_observation, reward, done, info
    